<a href="https://colab.research.google.com/github/hyulianton/JaringanSyarafTiruan/blob/main/jst_Generasi_Teks_dengan_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyek: Generasi Teks Karakter-Level dengan LSTM

Ini adalah implementasi dasar dari Recurrent Neural Network (RNN) dengan arsitektur Long Short-Term Memory (LSTM) untuk memprediksi karakter teks berikutnya. Tujuannya adalah melatih model agar dapat menghasilkan teks baru yang meniru gaya dan pola dari korpus teks pelatihan.

### Cell 1: Setup Lingkungan dan Import Library

Sel ini melakukan impor semua library yang diperlukan, yaitu NumPy untuk operasi array, dan TensorFlow/Keras untuk membangun serta melatih model jaringan saraf. Pengecekan GPU juga disertakan untuk optimasi performa.

In [1]:
# Import library yang diperlukan
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import random
import sys
import time

# Pengecekan Versi dan Ketersediaan GPU
# Penggunaan GPU sangat disarankan untuk pelatihan model Neural Network
print(f"Versi TensorFlow: {tf.__version__}")
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU tersedia dan akan digunakan: {gpu_devices}")
else:
    print("GPU tidak tersedia. TensorFlow akan menggunakan CPU.")

Versi TensorFlow: 2.19.0
GPU tersedia dan akan digunakan: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Cell 2: Persiapan Data Teks (Korpus)

Korpus teks adalah data mentah yang akan dipelajari oleh model. Dalam sel ini, kita memuat teks, mengubahnya menjadi huruf kecil, dan membuat dua *mapping* penting: dari karakter ke integer (`char_to_int`) dan sebaliknya (`int_to_char`).

**Konsep Kunci:** Pemetaan karakter ke integer diperlukan karena Neural Network hanya dapat memproses data numerik.

In [2]:
# --- 1. Persiapan Data (Load dan Preprocess) ---
print("--- Memuat dan Memproses Data ---")

# Korpus Teks Sederhana untuk demonstrasi.
# Untuk hasil yang lebih menarik, ganti teks ini dengan korpus yang lebih besar (misalnya novel).
text = """Hai teman-teman! Selamat datang di perkuliahan Jaringan Syaraf Tiruan.
Kali ini kita akan membahas Recurrent Neural Networks dan Long Short-Term Memory.
Semoga kalian semua enjoy dan memahami materi ini dengan baik.
Jangan sungkan bertanya jika ada yang kurang jelas, ya!
AI itu keren banget, bukan?"""

# Ubah teks menjadi huruf kecil semua
text = text.lower()
print(f'Panjang total korpus teks: {len(text)} karakter.')

# Temukan semua karakter unik yang ada di teks (vocabulary)
chars = sorted(list(set(text)))
print(f'Total karakter unik (vocabulary size): {len(chars)}')
# print(f'Karakter unik: {chars}') # Uncomment untuk melihat semua karakter unik

# Buat mapping dari karakter ke integer dan sebaliknya
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

--- Memuat dan Memproses Data ---
Panjang total korpus teks: 299 karakter.
Total karakter unik (vocabulary size): 29


### Cell 3: Pembentukan Sekuens Input dan Vectorisasi

Model LSTM dilatih menggunakan sekuens. Kita menggunakan teknik **Sliding Window** untuk membuat banyak pasangan input (urutan 40 karakter) dan target (karakter ke-41).

**One-Hot Encoding** dilakukan untuk mengubah data sekuensial karakter menjadi format tensor biner yang dapat diterima oleh layer LSTM.

In [3]:
# --- 2. Menyiapkan Data Pelatihan (Sequences) ---
print("\n--- Menyiapkan Data Sekuensial untuk Pelatihan ---")

SEQUENCE_LENGTH = 40 # Panjang sekuens input (konteks)
step = 3             # Jarak pergeseran window

sentences = []   # Input (X): sekuens 40 karakter
next_chars = []  # Output (y): karakter berikutnya

# Lakukan sliding window
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])

print(f'Jumlah sekuens pelatihan yang dibuat: {len(sentences)}')

# Vectorisasi data: Ubah karakter menjadi one-hot encoding
# X: (jumlah_sekuens, SEQUENCE_LENGTH, jumlah_karakter_unik)
# y: (jumlah_sekuens, jumlah_karakter_unik)
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

# Isi array X dan y
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        # One-hot encoding untuk input X
        X[i, t, char_to_int[char]] = 1
    # One-hot encoding untuk target y
    y[i, char_to_int[next_chars[i]]] = 1

print(f'Bentuk data input (X): {X.shape}')
print(f'Bentuk data target (y): {y.shape}')


--- Menyiapkan Data Sekuensial untuk Pelatihan ---
Jumlah sekuens pelatihan yang dibuat: 87
Bentuk data input (X): (87, 40, 29)
Bentuk data target (y): (87, 29)


### Cell 4: Membangun dan Mengkompilasi Model LSTM

Model dibangun menggunakan layer LSTM sebagai inti pemrosesan sekuens, diikuti oleh layer `Dense` dengan aktivasi `softmax` untuk menghasilkan distribusi probabilitas dari semua karakter unik.

In [4]:
# --- 3. Membangun Model LSTM ---
print("\n--- Membangun Model LSTM ---")

model = Sequential()
# Layer LSTM: 128 unit (kapasitas memori model)
# input_shape: (panjang_sekuens, jumlah_fitur_per_langkah_waktu)
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
# Layer Dense: Output sebanyak jumlah karakter unik (ukuran vocabulary)
model.add(Dense(len(chars)))
# Aktivasi Softmax: Mengubah output menjadi probabilitas
model.add(Activation('softmax'))

# Konfigurasi Optimizer dan Kompilasi
# RMSprop dengan learning_rate 0.01 adalah konfigurasi yang sering berhasil untuk RNN
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Tampilkan ringkasan arsitektur model
model.summary()


--- Membangun Model LSTM ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        80,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 29)             │         3,741 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 29)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,637 (330.61 KB)

 Trainable params: 84,637 (330.61 KB)

 Non-trainable params: 0 (0.00 B)

### Cell 5: Fungsi Pembantu untuk Sampling Karakter

Fungsi `sample` ini memperkenalkan unsur keacakan yang dikendalikan oleh variabel `temperature`. Ini memungkinkan model untuk menghasilkan teks yang lebih bervariasi dan tidak hanya mengulang karakter dengan probabilitas tertinggi, sehingga menghasilkan teks yang lebih alami (atau kreatif).


In [5]:
# --- Fungsi Pembantu untuk Sampling Karakter ---
def sample(preds, temperature=1.0):
    """
    Memilih karakter berikutnya berdasarkan probabilitas (preds) dan temperature.
    Temperature rendah = hasil konservatif. Temperature tinggi = hasil kreatif.
    """
    # Pastikan input adalah float
    preds = np.asarray(preds).astype('float64')

    # Aplikasikan Temperature
    # Teknik ini meningkatkan atau mengurangi perbedaan antar probabilitas
    preds = np.log(preds + 1e-8) / temperature # Tambah epsilon untuk mencegah log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) # Normalisasi probabilitas agar jumlahnya 1

    # Pilih satu karakter secara acak (multinomial) berdasarkan probabilitas yang dimodifikasi
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Cell 6: Pelatihan Model dan Generasi Teks Berkala

Sel ini berisi logika pelatihan utama dan menggunakan **Callback Keras** untuk secara berkala memanggil fungsi generasi teks. Callback ini memungkinkan kita melihat hasil keluaran model (*simulated output*) di tengah proses pelatihan.

In [6]:
# --- 4. Melatih Model dan Menghasilkan Teks ---
print("\n--- Memulai Pelatihan Model (Mohon ditunggu!) ---")

# Kelas Callback untuk Generasi Teks Real-time
class TextGenerationCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Generasi teks setiap 5 epoch dan di awal
        if (epoch + 1) % 10 == 0 or epoch == 1:
            print(f'\n=============================================================')
            print(f'--- Generasi Teks Setelah Epoch {epoch + 1} ---')
            print(f'=============================================================')

            # Ambil sekuens acak dari teks asli sebagai 'seed' (teks awal)
            start_index = random.randint(0, len(text) - SEQUENCE_LENGTH - 1)
            seed_text = text[start_index: start_index + SEQUENCE_LENGTH]

            # Coba dengan berbagai nilai temperature
            for temperature in [0.2, 0.5, 1.0]:
                print(f'\n--- Diversity (Temperature): {temperature} ---')
                generated_text = seed_text
                # sys.stdout.write digunakan agar output muncul di Colab/Jupyter secara real-time
                sys.stdout.write('Seed (Teks Awal): \n"' + generated_text + '"\nGenerated: \n')

                # Generasi 200 karakter selanjutnya
                for i in range(200):
                    # 1. Siapkan input untuk prediksi (one-hot encoding)
                    x_pred = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
                    for t, char in enumerate(generated_text):
                        if char in char_to_int:
                             x_pred[0, t, char_to_int[char]] = 1.
                        # Karakter yang tidak ada di vocabulary diabaikan

                    # 2. Prediksi probabilitas karakter berikutnya
                    # Gunakan try/except untuk menangani potential error saat model.predict
                    try:
                        preds = model.predict(x_pred, verbose=0)[0]
                    except Exception as e:
                        print(f"\n[ERROR] Prediksi gagal di langkah {i}: {e}")
                        break

                    # 3. Sample karakter berdasarkan probabilitas dan temperature
                    next_index = sample(preds, temperature)
                    next_char = int_to_char[next_index]

                    # 4. Update sekuens: Geser window (buang karakter pertama)
                    generated_text += next_char
                    generated_text = generated_text[1:]

                    sys.stdout.write(next_char)
                    sys.stdout.flush()
                sys.stdout.write('\n')

# Mulai pelatihan model!
# Dengan korpus kecil, 50 epoch sudah cukup untuk melihat konvergensi.
model.fit(X, y,
          batch_size=128, # Ukuran batch
          epochs=50,      # Jumlah epoch
          callbacks=[TextGenerationCallback()])

print("\nPelatihan model selesai! Periksa output di atas untuk melihat contoh hasil generasi teks.")


--- Memulai Pelatihan Model (Mohon ditunggu!) ---
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 3.3627
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 3.0329
--- Generasi Teks Setelah Epoch 2 ---

--- Diversity (Temperature): 0.2 ---
Seed (Teks Awal): "ada yang kurang jelas, ya!
ai itu keren "
Generated:  nann a an a  nnn  n  a  nnn a a    an a a an n  n a nn a a anna a nnaa a  na  nn  n a  a  nn  e a    nn n  n a n a a n an ann  an  ann   n n n n  na nnna  n a  aan e a  an ann a   a  a aaaa an    a  

--- Diversity (Temperature): 0.5 ---
Seed (Teks Awal): "ada yang kurang jelas, ya!
ai itu keren "
Generated: nn   naninnnnnna a a aaa n a  nn aa  ana nnn  n n nn aaenaa an  e nann   a na   nan  an ne  iaa na a nngn i an   i aai na  ea  n  na  an naa  aeana  n a na na an   a naan n   e inn    n  ea   n naae e

--- Diversity (Temperature): 1.0 ---
Seed (Teks Awal): "ada yang kurang jelas, ya!
ai itu keren "
Generated: ngaa nnaa eaeai  nna  ni ninaa anaaag gaeeaannnn ee a

### Analisis Contoh Output (Simulasi Hasil Pelatihan)

Setelah model selesai dilatih, Anda akan melihat serangkaian teks yang dihasilkan di dalam *output* Cell 6.

Penting untuk dicatat bahwa dengan korpus teks yang **sangat kecil** seperti yang kita gunakan, model cenderung **menghafal** (overfit) data asli, terutama pada *temperature* rendah. Namun, dengan *temperature* yang lebih tinggi, kita dapat mengamati upaya model untuk berkreasi:

| Temperature | Hasil yang Diharapkan | Keterangan |
| :---: | :--- | :--- |
| **0.2** | `...semoga kalian semua enjoy dan memahami materi ini dengan baik. jangan sungkan bertanya jika ada yang kurang jelas, ya! ai itu keren banget, bukan?` | Hasil akan sangat konservatif dan sering mengulang bagian teks asli yang paling sering muncul. Model sangat yakin dengan prediksinya. |
| **0.5** | `...dan long short-term memorr. semoga kalian semua enjoy dan memahami materi ini dengan baik. jangan sungkan brtanya jika ada yang kurang jelas, ya! a` | Model mulai melakukan sedikit kesalahan ejaan minor atau penggantian karakter (e.g., `memorr` atau `brtanya`), menunjukkan sedikit penyimpangan dari teks asli. |
| **1.0** | `...kalian smua enjoy dn mamahmi materi ini dgna bik. jagan sungkn bertny jiak ada yaag kuran jelas, ya! ai ittu kern bngat, bukan?` | Output menjadi lebih "kreatif" namun kurang koheren. Terjadi banyak *typo* (e.g., `dgna bik`, `jiak`, `bnagt`), menunjukkan bahwa model memilih dari probabilitas yang sangat rendah. |

Untuk tugas akademik selanjutnya, saya sarankan Anda mengganti teks di Cell 2 dengan korpus yang jauh lebih besar (misalnya 1MB-10MB teks) untuk melihat potensi sesungguhnya dari model LSTM dalam menghasilkan teks yang koheren dan unik.